In [1]:
# # 01 — E5-эмбеддинги (query_text & item_title) с сохранением
# - Модель: intfloat/multilingual-e5-small (384d)
# - Префиксы согласно e5: "query: ..." и "passage: ..."
# - Сохранение: `memmap` (float16) + индексы **или** parquet-шарды
# - Прогресс-бары: tqdm
#
# Требования:
# pip install polars pyarrow sentence-transformers torch tqdm


import os
import json
from typing import List, Tuple

import polars as pl
import numpy as np
import torch
from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer

C:\Users\idine\PycharmProjects\Avito_Test\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
BASE_DIR = os.path.dirname(os.getcwd())
DATA_DIR = os.path.join(BASE_DIR, "Data")
TRAIN_PATH = os.path.join(DATA_DIR, "train-dset.parquet")
TEST_PATH  = os.path.join(DATA_DIR, "test-dset-small.parquet")

In [8]:
MODEL_NAME = "intfloat/multilingual-e5-small"
EMB_DIM    = 384  # для base будет 768

# Режим сохранения: "memmap" или "parquet"
SAVE_MODE  = "memmap"

In [9]:
# Директории для сохранения
PARQUET_DIR = "embeddings_parquet"
MEMMAP_DIR  = "../Data/embeddings_memmap"

In [10]:
# Параметры батчинга/шардирования
BATCH_SIZE  = 1024          # для инференса модели
SHARD_SIZE  = 250_000       # размер шарда при parquet-выгрузке

# Рандом и девайс
SEED = 42
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


In [11]:
np.random.seed(SEED)
torch.manual_seed(SEED)

In [12]:
# ## 1) Загрузка уникальных текстов (train + test)
# - Уникализируем по `query_id` и `item_id`
# - Берём соответствующие тексты: `query_text`, `item_title`

# ленивые сканы
train_lf = pl.scan_parquet(TRAIN_PATH)
test_lf = pl.scan_parquet(TEST_PATH)

In [13]:
# уникальные по query_id
queries_uniques = (
    pl.concat([
        train_lf.select("query_id", "query_text"),
        test_lf.select("query_id", "query_text"),
    ])
    .unique(subset=["query_id"])
    .collect(streaming=True)
)


C:\Users\idine\AppData\Local\Temp\ipykernel_44180\1145090502.py:3: DeprecationWarning: the `streaming` parameter was deprecated in 1.25.0; use `engine` instead.
  pl.concat([


In [14]:
# уникальные по item_id
items_uniques = (
    pl.concat([
        train_lf.select("item_id", "item_title"),
        test_lf.select("item_id", "item_title"),
    ])
    .unique(subset=["item_id"])
    .collect(streaming=True)
)

C:\Users\idine\AppData\Local\Temp\ipykernel_44180\2638892594.py:3: DeprecationWarning: the `streaming` parameter was deprecated in 1.25.0; use `engine` instead.
  pl.concat([


In [15]:
print(f"[data] queries_uniques: {queries_uniques.shape}, items_uniques: {items_uniques.shape}")

[data] queries_uniques: (690695, 2), items_uniques: (5986464, 2)


In [16]:
# ## 2) Инициализация e5-модели и функция кодирования
# - Важно: e5 требует **префиксов** — `"query:"` и `"passage:"`
# - `normalize_embeddings=True` ⇒ L2-норма = 1, значит `dot = cosine`

model = SentenceTransformer(MODEL_NAME, device=DEVICE)

In [17]:
def encode_texts(texts: List[str], prefix: str, batch_size: int = BATCH_SIZE) -> np.ndarray:
    """Кодируем список строк e5, добавляя нужный префикс. Возвращаем float32 L2-нормированные вектора."""
    prepped = [f"{prefix} {t if t is not None else ''}".strip() for t in texts]
    embs = model.encode(
        prepped,
        batch_size=batch_size,
        show_progress_bar=False,   # прогресс дадим внешним tqdm
        normalize_embeddings=True, # L2-нормализация
        convert_to_numpy=True,
        device=DEVICE
    ).astype(np.float32)
    return embs

In [18]:
# ## 3A) Сохранение в Parquet (шарды)
# Каждая строка: `id` + `embedding` (List[Float32])
# - Плюсы: проще смотреть/джоинить
# - Минусы: больше места на диске

def to_parquet_shards(df_id_text: pl.DataFrame, id_col: str, text_col: str, prefix: str,
                      out_dir: str, shard_size: int = SHARD_SIZE):
    os.makedirs(out_dir, exist_ok=True)

    n = df_id_text.height
    n_shards = (n + shard_size - 1) // shard_size
    print(f"[parquet] {id_col}: {n} rows → {n_shards} shard(s)")

    # делаем детерминированный порядок по id (на всякий случай)
    df_id_text = df_id_text.sort(id_col)

    with tqdm(total=n, desc=f"encode+save [{id_col}]") as pbar:
        for i in range(n_shards):
            a, b = i * shard_size, min((i + 1) * shard_size, n)
            part = df_id_text.slice(a, b - a)
            texts = part.get_column(text_col).to_list()
            embs = encode_texts(texts, prefix=prefix, batch_size=BATCH_SIZE)  # (m, d)

            out = part.with_columns(pl.Series("embedding", [list(v) for v in embs]))
            out_path = os.path.join(out_dir, f"{id_col}_emb_shard_{i:03d}.parquet")
            out.write_parquet(out_path)
            pbar.update(b - a)
            tqdm.write(f"[parquet] saved shard {i + 1}/{n_shards}: {out_path} shape={out.shape}")


In [19]:
# ## 3B) Сохранение в np.memmap (float16) + индексы
# - Плюсы: экономия диска (≈×2), быстрый случайный доступ, удобно в тренинге
# - Сохраняем:
#   - бинарный `.memmap` массив (N × D, float16)
#   - `id2idx.json` — словарь для доступа по id

def to_memmap(df_id_text: pl.DataFrame, id_col: str, text_col: str, prefix: str,
              out_dir: str, dim: int = EMB_DIM, batch_size: int = BATCH_SIZE, fname_prefix: str = "item"):
    os.makedirs(out_dir, exist_ok=True)

    # детерминированный порядок по id
    df_id_text = df_id_text.sort(id_col)

    ids = df_id_text.get_column(id_col).to_list()
    texts = df_id_text.get_column(text_col).to_list()
    N = len(ids)

    mmap_path = os.path.join(out_dir, f"{fname_prefix}_embeddings.f16.memmap")
    idmap_path = os.path.join(out_dir, f"{fname_prefix}_id2idx.json")

    emb_mm = np.memmap(mmap_path, dtype="float16", mode="w+", shape=(N, dim))
    id2idx = {}

    with tqdm(total=N, desc=f"encode+memmap [{fname_prefix}]") as pbar:
        for a in range(0, N, batch_size):
            b = min(a + batch_size, N)
            embs = encode_texts(texts[a:b], prefix=prefix, batch_size=batch_size)  # float32 normalized
            emb_mm[a:b] = embs.astype(np.float16)
            # индекс
            for j, _id in enumerate(ids[a:b]):
                id2idx[_id] = a + j
            pbar.update(b - a)

    emb_mm.flush()
    with open(idmap_path, "w", encoding="utf-8") as f:
        json.dump(id2idx, f)

    print(f"[memmap] saved: {mmap_path} (shape=({N},{dim}), dtype=float16)")
    print(f"[memmap] index: {idmap_path} (size={len(id2idx)})")

In [20]:
# ## 4) Запуск сохранения (выберите режим)
# - `SAVE_MODE = "parquet"`: будут сохранены шарды для query и item
# - `SAVE_MODE = "memmap"`: будут сохранены 2 файла `.memmap` и 2 индекса `.json`

if SAVE_MODE == "parquet":
    os.makedirs(PARQUET_DIR, exist_ok=True)
    # queries → parquet
    to_parquet_shards(
        queries_uniques,
        id_col="query_id",
        text_col="query_text",
        prefix="query:",
        out_dir=os.path.join(PARQUET_DIR, "queries"),
        shard_size=SHARD_SIZE
    )
    # items (titles) → parquet
    to_parquet_shards(
        items_uniques,
        id_col="item_id",
        text_col="item_title",
        prefix="passage:",
        out_dir=os.path.join(PARQUET_DIR, "items"),
        shard_size=SHARD_SIZE
    )

elif SAVE_MODE == "memmap":
    os.makedirs(MEMMAP_DIR, exist_ok=True)
    # queries → memmap
    to_memmap(
        queries_uniques,
        id_col="query_id",
        text_col="query_text",
        prefix="query:",
        out_dir=MEMMAP_DIR,
        dim=EMB_DIM,
        batch_size=BATCH_SIZE,
        fname_prefix="query"
    )
    # items (titles) → memmap
    to_memmap(
        items_uniques,
        id_col="item_id",
        text_col="item_title",
        prefix="passage:",
        out_dir=MEMMAP_DIR,
        dim=EMB_DIM,
        batch_size=BATCH_SIZE,
        fname_prefix="item"
    )
else:
    raise ValueError("SAVE_MODE must be 'parquet' or 'memmap'")


encode+memmap [query]: 100%|██████████| 690695/690695 [06:55<00:00, 1663.56it/s]


[memmap] saved: embeddings_memmap\query_embeddings.f16.memmap (shape=(690695,384), dtype=float16)
[memmap] index: embeddings_memmap\query_id2idx.json (size=690695)


encode+memmap [item]: 100%|██████████| 5986464/5986464 [1:33:10<00:00, 1070.79it/s] 


[memmap] saved: embeddings_memmap\item_embeddings.f16.memmap (shape=(5986464,384), dtype=float16)
[memmap] index: embeddings_memmap\item_id2idx.json (size=5986464)
